Request URL: http://www.cine21.com/rank/person/content
Request Method : POST 

section: actor
period_start: 2019-12
gender: all
page: 1

In [35]:
from bs4 import BeautifulSoup
import requests
import pymongo

In [36]:
conn = pymongo.MongoClient()
actor_db= conn.cine21
actor_collection = actor_db.actor_collection

In [37]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2019-12'
post_data['gender']= 'all'
post_data['page'] = 1

res = requests.post(cine21_url,data = post_data)

In [38]:
soup = BeautifulSoup(res.content,'html.parser')

In [39]:
actors = soup.select('li.people_li>div.name')
# li.people >div.name

In [40]:
for i in actors:
    print(i.text.split('(')[0])

이병헌
하정우
황정민
이정재
박정민
전혜진
이성민


In [53]:
import re
for i in actors:
    print(re.sub('\(\w*\)','',i.text))

이병헌
하정우
황정민
이정재
박정민
전혜진
이성민


In [44]:
for i in actors:
    print(i)

<div class="name"><a href="/db/person/info/?person_id=947">이병헌(6편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=21119">하정우(9편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=14268">황정민(5편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=1075">이정재(6편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=74826">박정민(10편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=11198">전혜진(4편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=25123">이성민(6편)</a></div>


In [114]:
actors = soup.select('li.people_li>div.name')
actor_detail_info = list()
for i in actors:
    alink = 'http://www.cine21.com/'+i.select_one('a').attrs['href']
    response_actor = requests.get(alink)
    soup_a = BeautifulSoup(response_actor.content,'html.parser')
    people_name = soup_a.select('div.people_info>p.name')
    default_info = soup_a.select_one('ul.default_info')
    actor_detail = default_info.select('li')
    actor_info_dict={}
    for a in people_name:
        a_1 = a.text.strip()
        a_1 = re.sub('<.*?>','',str(a))
        a_1 = a_1.replace('\t','').replace('\r','').replace('\n',' ')
#         a_1 = re.sub('<span.*?>.*?</span>','',str(a))
        actor_info_dict['이름'] = a_1
#         print(a_1)
#         print(a.text)
    
    for actor_item in actor_detail:
#         print(actor_item)
        actor_item_field = actor_item.select_one('span.tit').text
#         print(actor_item_field)
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
#         print(actor_item_value)
        actor_item_value = re.sub('<.*?>','',actor_item_value)
#         print(actor_item_value)
        actor_info_dict[actor_item_field]=actor_item_value
    actor_detail_info.append(actor_info_dict)


## greedy(.*) vs Non-Greedy(.*?)
https://regexr.com
-<li><span class="tit">성별</span>남</li>
.문자는 줄바꿈 문자인 \n을 제외한 모든 문자 한계를 의미함
* 은 앞문자가 0번 또는 그이상 반복되는 패턴 
.* 

In [115]:
actor_detail_info

[{'이름': '이병헌  (Lee Byunghun) ',
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'이름': '하정우  (Ha Jungwoo) ',
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'이름': '황정민  (Hwang Jungmin) ',
  '직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'이름': '이정재  (Lee Jungjae) ',
  '직업': '배우',
  '성별': '남',
  '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n',
  '소속사': '아티스트컴퍼니'},
 {'이름': '박정민  (PARK Jeong-min) ',
  '직업': '배우',
  '생년월일': '1987-02-25',
  '성별': '남',
  '신장/체중': '178cm, 63kg',
  '학교': '한국예술종합학교 영상원 연극원 연기과'},
 {'이름': '전혜진  (Jeon Hyejin)

In [ ]:
# 흥행 지수,출연 영화 추출

In [116]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2019-12'
post_data['gender']= 'all'
post_data['page'] = 1

res = requests.post(cine21_url,data = post_data)
soup = BeautifulSoup(res.content,'html.parser')

In [151]:
actors = soup.select('li.people_li>div.name')  
hits= soup.select('ul.num_info>li>strong')
# print(actors)
# print(hits)
movies = soup.select('ul.mov_list')

actors = soup.select('li.people_li>div.name')
actor_detail_info = list()
for index, i in enumerate(actors):
    actor_name= re.sub('\(\w*\)','',i.text)
#     print("이름",re.sub('\(\w*\)','',i.text))
    actor_hints = int(hits[index].text.replace(',','')) 
    movie_titles = movies[index].select('li a span')
    movie_list =[]
    for mt in movie_titles:
        movie_list.append(mt.text)
    print(movie_list)
    actor_info_dict={}
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hints
    actor_info_dict['출연영화'] = movie_list
    
    ##################################
    alink = 'http://www.cine21.com/'+i.select_one('a').attrs['href']
    response_actor = requests.get(alink)
    soup_a = BeautifulSoup(response_actor.content,'html.parser')
    default_info = soup_a.select_one('ul.default_info')
    actor_detail = default_info.select('li')
    
    
    for actor_item in actor_detail:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
        actor_item_value = re.sub('<.*?>','',actor_item_value)
        if actor_item_field == '홈페이지':
            actor_item_value = re.sub('\n','',actor_item_value)
        else:
            pass
        actor_info_dict[actor_item_field]=actor_item_value
    actor_detail_info.append(actor_info_dict)


['백두산', '남산의 부장들', '그것만이 내 세상', '공동경비구역 JSA', '내부자들', '광해, 왕이 된 남자']
['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '신과 함께-인과 연']
['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라']
['다만 악에서 구하소서', '신세계', '인천상륙작전', '사바하', '암살', '도둑들']
['시동', '다만 악에서 구하소서', '사냥의 시간', '동주', '그것만이 내 세상', '들개']
['백두산', '불한당: 나쁜 놈들의 세상', '뺑반', '사도']
['남산의 부장들', '미스터 주: 사라진 VIP', '부당거래', '밀양', '뺑반', '변호인']


In [153]:
actor_detail_info

[{'배우이름': '이병헌',
  '흥행지수': 72950,
  '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '공동경비구역 JSA', '내부자들', '광해, 왕이 된 남자'],
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': 'http://www.leebyunghun.kr/',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'배우이름': '하정우',
  '흥행지수': 49623,
  '출연영화': ['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '신과 함께-인과 연'],
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/ft.hajungwoo',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'배우이름': '황정민',
  '흥행지수': 43701,
  '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'],
  '직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'배우이름': '이정재',
  '흥행지수': 29134,
  '출연영화': ['다만 악에서 구하소서', '신세계', '인천상륙작전', '사바하', '암살', '도둑들'],
  '직업': '배우',

In [ ]:
#### 배우 상세정보 추출하기 통합본
import pymongo
conn = pymongo.MongoClient()
actor_db= conn.cine21
actor_collection = actor_db.actor_collection

In [172]:
from bs4 import BeautifulSoup
import requests
import re
actor_detail_info = list()
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2019-12'
post_data['gender']= 'all'
for num in range(1,21):
    
    post_data['page'] = num
    
    res = requests.post(cine21_url,data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    actors = soup.select('li.people_li>div.name')  
    hits= soup.select('ul.num_info>li>strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li>span.grade')
    for index, i in enumerate(actors):
        actor_name= re.sub('\(\w*\)','',i.text)
    #     print("이름",re.sub('\(\w*\)','',i.text))
        actor_hints = int(hits[index].text.replace(',','')) 
        movie_titles = movies[index].select('li a span')
        movie_list =[]
        for mt in movie_titles:
            movie_list.append(mt.text)
        print(movie_list)
        actor_info_dict={}
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hints
        actor_info_dict['출연영화'] = movie_list
        actor_info_dict['랭킹'] = rankings[index].text

        
        ##################################
        alink = 'http://www.cine21.com/'+i.select_one('a').attrs['href']
        response_actor = requests.get(alink)
        soup_a = BeautifulSoup(response_actor.content,'html.parser')
        default_info = soup_a.select_one('ul.default_info')
        actor_detail = default_info.select('li')
        
        
        for actor_item in actor_detail:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
            actor_item_value = re.sub('<.*?>','',actor_item_value)
            if actor_item_field == '홈페이지':
                actor_item_value = re.sub('\n','',actor_item_value)
            else:
                pass
            actor_info_dict[actor_item_field]=actor_item_value
        actor_detail_info.append(actor_info_dict)
actor_collection.insert_many(actor_detail_info)

['백두산', '남산의 부장들', '그것만이 내 세상', '공동경비구역 JSA', '내부자들', '광해, 왕이 된 남자']
['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '암살']
['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라']
['다만 악에서 구하소서', '신세계', '인천상륙작전', '사바하', '도둑들', '암살']
['시동', '다만 악에서 구하소서', '사냥의 시간', '동주', '그것만이 내 세상', '들개']
['백두산', '불한당: 나쁜 놈들의 세상', '뺑반', '사도']
['남산의 부장들', '미스터 주: 사라진 VIP', '부당거래', '밀양', '뺑반', '변호인']
['#살아있다', '소리도 없이', '버닝', '사도', '우아한 거짓말']
['남산의 부장들', '강철비2: 정상회담', '국제수사', '무뢰한', '변호인', '강철비']
['백두산', '건축학개론']
['반도', '1987']
['죽지않는 인간들의 밤', '두번할까요', '반도', '명량']
['남산의 부장들', '오! 문희', '환상속의 그대', '미성년', '미쓰백', '1987']
['시동', '강철비2: 정상회담', '미성년', '뺑반', '오래된 정원']
['담보', '두번할까요', '수상한 그녀', '우아한 거짓말']
['시동', '피원에이치: 새로운 세계의 시작', '장수상회']
['반도', '너의 이름은.', '증인', '개를 훔치는 완벽한 방법', '소원']
['도굴', '사냥의 시간', '건축학개론', '아이 캔 스피크', '파수꾼', '박열']
['#살아있다', '7번방의 선물', '뷰티 인사이드', '시라노; 연애조작단']
['도망친 여자', '메기', '후쿠오카', '나의 특별한 형제', '반도', '사이비']
['시동', '삼진그룹 영어토익반', '나를 찾아줘', '밀양', '어제 일은 모두 괜찮아', '증인']
['담보', '동감']
['시동']
['시동', 

In [162]:
actor_detail_info

[{'배우이름': '이병헌',
  '흥행지수': 72950,
  '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '공동경비구역 JSA', '내부자들', '광해, 왕이 된 남자'],
  '랭킹': '1',
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': 'http://www.leebyunghun.kr/',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'배우이름': '하정우',
  '흥행지수': 49623,
  '출연영화': ['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '신과 함께-인과 연'],
  '랭킹': '2',
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/ft.hajungwoo',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'배우이름': '황정민',
  '흥행지수': 43701,
  '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'배우이름': '이정재',
  '흥행지수': 29134,
  '출연영화': ['다만 악에서 구하소서', '신세계', '인천상륙

In [163]:
conn = pymongo.MongoClient()
actor_db= conn.cine21
actor_collection = actor_db.actor_collection

In [ ]:
actor_collection.insert_many(actor_detail_info)

In [171]:
actor_collection.delete_many({})

{'_id': ObjectId('5fd07e69515110a083eacd87'), '배우이름': '이병헌', '흥행지수': 72950, '출연영화': ['백두산', '남산의 부장들', '그것만이 내 세상', '공동경비구역 JSA', '내부자들', '광해, 왕이 된 남자'], '랭킹': '1', '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': 'http://www.leebyunghun.kr/', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름'}
 
{'_id': ObjectId('5fd07e69515110a083eacd88'), '배우이름': '하정우', '흥행지수': 49623, '출연영화': ['백두산', '클로젯', '멋진 하루', '아가씨', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '신과 함께-인과 연'], '랭킹': '2', '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': 'https://www.facebook.com/ft.hajungwoo', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}
 
{'_id': ObjectId('5fd07e69515110a083eacd89'), '배우이름': '황정민', '흥행지수': 43701, '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'], '랭킹': '3', '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트'}
 
{'_